In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../")
sys.path.append("../../")

In [ ]:
from utils.common_settings import *
from utils.basic_utils import utils_tex
from data_loader import data_loader_ml, data_loader_dl
from utils import path_definitions
from utils.cv_split import judge_corner_cvsplit
from algorithm.base import DepressionDetectionAlgorithmBase
from data_loader.data_loader_ml import DatasetDict, DataRepo
from utils import train_eval_pipeline

In [ ]:
import matplotlib as mpl
from matplotlib import rcParams
mpl.rcParams.update(mpl.rcParamsDefault)

plt.style.use(['science', "grid"])

plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
ds_keys = ["INS-W_1", "INS-W_2", "INS-W_3", "INS-W_4"]
colors = [ "#1a9641", "#2b83ba", "#fdae61", "#d7191c", "#ffffbf"]

In [ ]:
model_ml = ['ml_canzian', 'ml_saeb', 'ml_farhan', 'ml_wahle', 'ml_lu', 'ml_wang', 'ml_xu_interpretable', 'ml_xu_personalized', 'ml_chikersal']
model_dl = ['dl_erm_1dCNN', 'dl_erm_2dCNN', 'dl_erm_LSTM', 'dl_erm_Transformer', 'dl_erm_mixup', 'dl_irm',
 'dl_dann_ds_as_domain', 'dl_dann_person_as_domain', 'dl_csd_ds_as_domain', 'dl_csd_person_as_domain',
 'dl_mldg_ds_as_domain','dl_mldg_person_as_domain', 'dl_masf_ds_as_domain', 'dl_masf_person_as_domain',
 'dl_siamese', 'dl_reorder']

model_list = model_ml + model_dl

In [ ]:
model_rename_display = {
    'ml_saeb': "Saeb et al.",
    'ml_canzian': "Canzian et al.",
    'ml_wahle': "Wahle et al.",
    'ml_farhan': "Farhan et al.", 
    'ml_lu': "Lu et al.", 
    'ml_wang' : "Wang et al.", 
    'ml_chikersal': "Chikersal et al.", 
    'ml_xu_interpretable': "Xu et al.-I", 
    'ml_xu_personalized': 'Xu et al.-P',
    'dl_erm_1dCNN': "ERM-1dCNN", 
    'dl_erm_2dCNN': "ERM-2dCNN", 
    'dl_erm_LSTM': "ERM-LSTM", 
    'dl_erm_Transformer': "ERM-Transformer", 
    'dl_erm_mixup': "ERM-Mixup", 
    'dl_irm': "IRM",
    'dl_mldg_ds_as_domain': "MLDG-D",
    'dl_mldg_person_as_domain': "MLDG-P",
    'dl_masf_ds_as_domain': "MASF-D",
    'dl_masf_person_as_domain': "MASF-P",
    'dl_csd_ds_as_domain': "CSD-D",
    'dl_csd_person_as_domain': "CSD-P",
    'dl_dann_ds_as_domain': "DANN-D",
    'dl_dann_person_as_domain': "DANN-P",
    'dl_siamese': "Siamese Network",
    'dl_reorder': "Reorder"
}

model_rename_latex = {
    'ml_saeb': "Saeb \\etal~\\cite{saeb_mobile_2015}",
    'ml_canzian': "Canzian \\etal~\\cite{canzian_trajectories_2015}",
    'ml_wahle': "Wahle \\etal~\\cite{wahle_mobile_2016}",
    'ml_farhan': "Farhan \\etal~\\cite{farhan_behavior_2016}",
    'ml_lu': "Lu \\etal~\\cite{lu_joint_2018}",
    'ml_wang' : "Wang \\etal~\\cite{wang_tracking_2018}", 
    'ml_chikersal': "Chikersal \\etal~\\cite{chikersal_detecting_2021}",
    'ml_xu_interpretable': "Xu \\etal-I~\\cite{xu_leveraging_2019}",
    'ml_xu_personalized': 'Xu \\etal-P~\\cite{xu_leveraging_2021}',
    'dl_erm_1dCNN': "ERM-1dCNN~\\cite{vapnik1999overview}", 
    'dl_erm_2dCNN': "ERM-2dCNN~\\cite{vapnik1999overview}", 
    'dl_erm_LSTM': "ERM-LSTM~\\cite{vapnik1999overview}", 
    'dl_erm_Transformer': "ERM-Transformer~\\cite{vapnik1999overview}", 
    'dl_erm_mixup': "ERM-Mixup~\\cite{zhang_mixup_2018}",
    'dl_irm': "IRM~\\cite{arjovsky_invariant_2020}",
    'dl_mldg_ds_as_domain': "MLDG-D~\\cite{li_learning_2017}",
    'dl_mldg_person_as_domain': "MLDG-P~\\cite{li_learning_2017}",
    'dl_masf_ds_as_domain': "MASF-D~\\cite{dou_domain_2019}",
    'dl_masf_person_as_domain': "MASF-P~\\cite{dou_domain_2019}",
    'dl_csd_ds_as_domain': "CSD-D~\\cite{piratla_efficient_2020}",
    'dl_csd_person_as_domain': "CSD-P~\\cite{piratla_efficient_2020}",
    'dl_dann_ds_as_domain': "DANN-D~\\cite{csurka_domain-adversarial_2017}",
    'dl_dann_person_as_domain': "DANN-P~\\cite{csurka_domain-adversarial_2017}",
    'dl_siamese': "Siamese Network~\\cite{koch_siamese_2015}",
    'dl_reorder': "Reorder~\\cite{xu_globem_2022}"
}

In [ ]:
evaluation_output_folder = "../evaluation_output/example_output/"

# Model Performance Analysis

In [ ]:
def get_results_table_ds(df_results, pred_target, metric, model_order = None, ds_order = None):
    df_pivot = utils_operation.get_df_rows(df_results, {"pred_target": pred_target}).pivot(
        index = "model", columns = "ds_test", values = metric)
    df_pivot["Avg"] = df_pivot.mean(axis = 1)
    if (model_order is None):
        return df_pivot
    else:
        return df_pivot.loc[model_order, ds_order + ["Avg"]]

In [ ]:
def get_pivot_table(df_results, ds_keys, print_latex=False, bold_last_row = False):
    df_pivot_balacc = get_results_table_ds(df_results, "dep_weekly", "test_balanced_acc",
        model_order=model_list,ds_order=ds_keys)
    df_pivot_rocauc = get_results_table_ds(df_results, "dep_weekly", "test_roc_auc",
        model_order=model_list,ds_order=ds_keys)
    df_pivot = pd.concat([df_pivot_balacc,df_pivot_rocauc], axis = 1)
    df_pivot_latex = df_pivot.rename(index = model_rename_latex).reset_index()
    df_pivot_latex = df_pivot_latex.rename({"model":"Model"}, axis = 1)
    if (print_latex):
        if bold_last_row:
            latex_str_list = utils_tex.table2tex(df_pivot_latex,bold_row_idx=len(df_pivot_latex)-1)
        else:
            latex_str_list = utils_tex.table2tex(df_pivot_latex)
        print("\n& ".join(latex_str_list))
    return df_pivot_balacc, df_pivot_rocauc

## One DS within Users

In [ ]:
folder_path = evaluation_output_folder+ "evaluation_single_dataset_within_user/dep_weekly/"
results_files = glob.glob(folder_path + "/*.pkl", recursive=True)

In [ ]:
df_results_singleds_within_user_list = []
for results_file in results_files:
    folder, file = os.path.split(results_file)
    model_name = file.replace(".pkl", "")

    # print(model_name)
    with open(results_file, 'rb') as f:
        results = pickle.load(f)
    for pred_target, results_dict in results['results_repo'].items():
        for ds_test, results_dict_test in results_dict.items():
            d = {"ds_test": ds_test, "pred_target": pred_target, "model": model_name}
            # if (model_name.startswith("ml") or "2dCNN" in model_name):
            d.update(results_dict_test)
            # else:
                # d.update(results_dict_test[ds_test])
            df_results_singleds_within_user_list.append(d)

df_results_singleds_within_user_list = pd.DataFrame(df_results_singleds_within_user_list)

In [ ]:
df_pivot_balacc_singleds, df_pivot_rocauc_singleds = get_pivot_table(
    df_results=df_results_singleds_within_user_list, ds_keys=ds_keys,
    print_latex=True)

## All but One DS

In [ ]:
folder_path= evaluation_output_folder + "/evaluation_allbutone_datasets/dep_weekly/"
results_files = glob.glob(folder_path + "/*.pkl", recursive=True)

In [ ]:
df_results_allbutoneds_list = []
for results_file in results_files:
    folder, file = os.path.split(results_file)
    model_name = file.replace(".pkl", "")
    if (model_name.startswith("dl_")):
        if (model_name.endswith("on_test")):
            continue
        else:
            model_name = model_name.replace("_direct", "")
    # print(model_name)
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

    for pred_target, results_dict in results['results_repo'].items():
        for ds_test, results_dict_test in results_dict.items():
            d = {"ds_test": ds_test, "pred_target": pred_target, "model": model_name}
            d.update(results_dict_test)
            df_results_allbutoneds_list.append(d)

df_results_allbutoneds_list = pd.DataFrame(df_results_allbutoneds_list)

In [ ]:
df_pivot_balacc_allbutoneds, df_pivot_rocauc_allbutoneds = get_pivot_table(
    df_results=df_results_allbutoneds_list, ds_keys=ds_keys,
    print_latex=True, bold_last_row=False)

In [ ]:
fig, axes = plt.subplots(figsize=(10,5.5), ncols = 2)

df_pivot_balacc = df_pivot_balacc_allbutoneds
df_pivot_rocauc = df_pivot_rocauc_allbutoneds

ax = axes[0]
keys = ds_keys
df_pivot_balacc = df_pivot_balacc.rename(index = model_rename_display)
df_pivot_balacc.index.name = ""
df_err = df_pivot_balacc[keys].std(axis=1) / 4
df_pivot_balacc[keys].mean(axis = 1).sort_values(ascending = True).plot.barh(
    ax=ax,xerr= df_err, color = colors[0],
    error_kw=dict(lw=2, capsize=3, capthick=1)
)
ax.set_xlim(0.44,0.61)
ax.set_title("Balanced Accuracy")

ax = axes[1]
keys = ds_keys
df_pivot_rocauc = df_pivot_rocauc.rename(index = model_rename_display)
df_pivot_rocauc.index.name = ""
df_err = df_pivot_rocauc[keys].std(axis=1) / 4
# df_pivot_rocauc[keys].mean(axis = 1).apply(lambda x : round(x,3)).reset_index()\
    # .rename({"":"model",0:"result"},axis=1).sort_values(['result', "model"], ascending = [True, False]).set_index("model").plot.barh(
df_pivot_rocauc[keys].mean(axis = 1).sort_values(ascending = True).plot.barh(
    ax=ax,xerr= df_err, color = colors[1],
    error_kw=dict(lw=2, capsize=3, capthick=1)
)
# ax.get_legend().remove()
ax.set_xlim(0.44,0.61)
ax.set_title("ROC AUC")

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0)


plt.tight_layout()
# plt.savefig("plots/results_allbutoneds_allmethods_weekly_neurips.png", dpi = 300, bbox_inches = "tight")
plt.show()

## Cross Group

In [ ]:
folder_path= evaluation_output_folder + "/evaluation_crosscovid_datasets/dep_weekly/"
results_files = glob.glob(folder_path + "/*.pkl", recursive=True)

In [ ]:
df_results_crosscovid_list = []
for results_file in results_files:
    folder, file = os.path.split(results_file)
    model_name = file.replace(".pkl", "")
    if (model_name.startswith("dl_")):
        if (model_name.endswith("on_test")):
            continue
        else:
            model_name = model_name.replace("_direct", "")
    # print(model_name)
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

    for pred_target, results_dict in results['results_repo'].items():
        for ds_test, results_dict_test in results_dict.items():
            d = {"ds_test": ds_test, "pred_target": pred_target, "model": model_name}
            d.update(results_dict_test)
            df_results_crosscovid_list.append(d)

df_results_crosscovid_list = pd.DataFrame(df_results_crosscovid_list)

In [ ]:
df_pivot_balacc_crosscovid, df_pivot_rocauc_crosscovid = get_pivot_table(
    df_results=df_results_crosscovid_list, ds_keys=["INS-W_1:INS-W_2", "INS-W_3:INS-W_4"],
    print_latex=True, bold_last_row=False)

In [ ]:
fig, axes = plt.subplots(figsize=(10,5.5), ncols = 2)

ax = axes[0]
keys = ["INS-W_3:INS-W_4", "INS-W_1:INS-W_2"]
df_pivot_balacc = df_pivot_balacc_crosscovid.rename(index = model_rename_display)
df_pivot_balacc.index.name = ""
df_err = df_pivot_balacc[keys].std(axis=1) / 2
df_pivot_balacc[keys].mean(axis = 1).sort_values(ascending = True).plot.barh(
    ax=ax,xerr= df_err, color = colors[2],
    error_kw=dict(lw=2, capsize=3, capthick=1)
)
ax.set_xlim(0.44,0.61)
ax.set_title("Balanced Accuracy")

ax = axes[1]
keys = ["INS-W_3:INS-W_4", "INS-W_1:INS-W_2"]
df_pivot_rocauc = df_pivot_rocauc_crosscovid.rename(index = model_rename_display)
df_pivot_rocauc.index.name = ""
df_err = df_pivot_rocauc[keys].std(axis=1) / 2
# df_pivot_rocauc[keys].mean(axis = 1).apply(lambda x : round(x,3)).reset_index()\
    # .rename({"":"model",0:"result"},axis=1).sort_values(['result', "model"], ascending = [True, False]).set_index("model").plot.barh(
df_pivot_rocauc[keys].mean(axis = 1).sort_values(ascending = True).plot.barh(
    ax=ax,xerr= df_err, color = colors[3],
    error_kw=dict(lw=2, capsize=3, capthick=1)
)
# ax.get_legend().remove()
ax.set_xlim(0.44,0.61)
ax.set_title("ROC AUC")

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0)


plt.tight_layout()
# plt.savefig("plots/results_allbutoneds_allmethods_weekly_neurips.png", dpi = 300, bbox_inches = "tight")
plt.show()

## Overlap

In [ ]:
folder_path= evaluation_output_folder + "evaluation_two_datasets_overlap/dep_weekly"
results_files = glob.glob(folder_path + "/*.pkl", recursive=True)

In [ ]:
df_results_twods_overlap_list = []
for results_file in results_files:
    folder, file = os.path.split(results_file)
    model_name = file.replace(".pkl", "")
    if (model_name.startswith("dl_")):
        if (model_name.endswith("on_test")):
            continue
        else:
            model_name = model_name.replace("_direct", "")
    # print(model_name)
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

    for pred_target, results_dict in results['results_repo'].items():
        for ds_test, results_dict_test in results_dict.items():
            results_dict_test_inner = results_dict_test[list(results_dict_test.keys())[0]]
            d = {"ds_test": ds_test, "pred_target": pred_target, "model": model_name}
            d.update(results_dict_test_inner)
            df_results_twods_overlap_list.append(d)

df_results_twods_overlap_list = pd.DataFrame(df_results_twods_overlap_list)

In [ ]:
df_pivot_balacc_overlap, df_pivot_rocauc_overlap = get_pivot_table(
    df_results=df_results_twods_overlap_list, ds_keys=ds_keys,
    print_latex=True, bold_last_row=False)

In [ ]:
fig, axes = plt.subplots(figsize=(10,5.5), ncols = 2)

ax = axes[0]
keys = ds_keys
df_pivot_balacc = df_pivot_balacc_overlap.rename(index = model_rename_display)
df_pivot_balacc.index.name = ""
df_err = df_pivot_balacc[keys].std(axis=1) / 4
df_pivot_balacc[keys].mean(axis = 1).sort_values(ascending = True).plot.barh(
    ax=ax,xerr= df_err, color = colors[2],
    error_kw=dict(lw=2, capsize=3, capthick=1)
)
ax.set_xlim(0.44,0.7)
ax.set_title("Balanced Accuracy")

ax = axes[1]
keys = ds_keys
df_pivot_rocauc = df_pivot_rocauc_overlap.rename(index = model_rename_display)
df_pivot_rocauc.index.name = ""
df_err = df_pivot_rocauc[keys].std(axis=1) / 4
# df_pivot_rocauc[keys].mean(axis = 1).apply(lambda x : round(x,3)).reset_index()\
    # .rename({"":"model",0:"result"},axis=1).sort_values(['result', "model"], ascending = [True, False]).set_index("model").plot.barh(
df_pivot_rocauc[keys].mean(axis = 1).sort_values(ascending = True).plot.barh(
    ax=ax,xerr= df_err, color = colors[3],
    error_kw=dict(lw=2, capsize=3, capthick=1)
)
# ax.get_legend().remove()
ax.set_xlim(0.44,0.7)
ax.set_title("ROC AUC")

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0)


plt.tight_layout()
# plt.savefig("plots/results_allbutoneds_allmethods_weekly_neurips.png", dpi = 300, bbox_inches = "tight")
plt.show()

## Merge all results

In [ ]:
df_pivot = pd.concat([df_pivot_balacc_singleds[["Avg"]],
                      df_pivot_balacc_allbutoneds[["Avg"]],
                      df_pivot_balacc_crosscovid[["Avg"]],
                      df_pivot_balacc_overlap[["Avg"]],
                     ], axis = 1)

In [ ]:
df_pivot_sem = pd.concat([df_pivot_balacc_singleds.iloc[:,:df_pivot_balacc_singleds.shape[-1]-1].std(axis = 1)/2,
                      df_pivot_balacc_allbutoneds.iloc[:,:df_pivot_balacc_allbutoneds.shape[-1]-1].std(axis = 1)/2,
                      df_pivot_balacc_crosscovid.iloc[:,:df_pivot_balacc_crosscovid.shape[-1]-1].std(axis = 1)/np.sqrt(2),
                      df_pivot_balacc_overlap.iloc[:,:df_pivot_balacc_overlap.shape[-1]-1].std(axis = 1)/2,
                     ], axis = 1)

In [ ]:
df_pivot_final = []
for idx, row in df_pivot.iterrows():
    row_std = df_pivot_sem.loc[idx]
    row_3 = row.apply(lambda x : f"{x:.3f}")
    row_std_3 = row_std.apply(lambda x : f"{x:.3f}")
    row_new = []
    for x,y in zip(row_3, row_std_3):
        row_new.append(f"{x}{{\\small $\\pm${y}}}")
    df_pivot_final.append(row_new)
df_pivot_final = pd.DataFrame(df_pivot_final, index=df_pivot.index)

In [ ]:
df_pivot_latex = df_pivot_final.rename(index = model_rename_latex).reset_index()
df_pivot_latex.columns = ["Model"] + ["Avg"] * (df_pivot_latex.shape[1]-1)
latex_str_list = utils_tex.table2tex(df_pivot_latex)
print("\n& ".join(latex_str_list))